In [7]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
# 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows의 경우 '맑은 고딕'
# Mac의 경우: plt.rcParams['font.family'] = 'AppleGothic'
# Linux의 경우 적절한 한글 폰트를 설치해야 함 (예: 'NanumGothic')

# 음수 기호가 깨지는 문제 해결 (minus 폰트 설정)
plt.rcParams['axes.unicode_minus'] = False

In [9]:

# 엑셀 파일 불러오기
# 파일 경로를 지정하세요.
merged_df = pd.read_excel('merged_file.xlsx')  # A 데이터
df = pd.read_excel('df_b.xlsx')  # B 데이터

In [10]:
# 열 이름에서 _x000D_ 제거
merged_df.columns = merged_df.columns.str.replace('_x000D_', '', regex=True)
df.columns = df.columns.str.replace('_x000D_', '', regex=True)

In [11]:
# A데이터 열에서 각 행의 최댓값을 찾아 Max_Value 열로 추가
merged_df['RSB_최종속도'] = merged_df[['RSB_STD_SPEED_20','RSB_STD_SPEED_21','RSB_STD_SPEED_22','RSB_STD_SPEED_23','RSB_STD_SPEED_24','RSB_STD_SPEED_25']].max(axis=1)
merged_df['RSM(LPM)'] = merged_df['RSM(SM)(LPM)']+merged_df['RSM(RM)(LPM)']

In [12]:
# '압연년월' 열을 datetime 형식으로 변환 (B 데이터에 필요)
df['압연년월'] = pd.to_datetime(df['압연년월']).dt.strftime('%Y-%m')  # '날짜' 열에서 년월을 문자열로 변환

# B 데이터 그룹화하여 불량율 계산
grouped = df.groupby(['외경', '압연년월']).agg({
    '투입중량': 'sum',
    '발생중량': 'sum'
}).reset_index()

# 불량율 계산
grouped['불량율'] = (grouped['발생중량'] / grouped['투입중량']) * 100

# 고유한 '외경' 값 가져오기
unique_conditions = grouped['외경'].unique()

# A 그래프에서 사용할 y값 목록
y_columns = ['균열대.1', '가열대.1','가열대_분','균열대_분','총재로_분','인덕션히터후면_온도']
# 소수점(0f)_온도: y값으로 사용할 열들을 추가('균열대.1', '가열대.1','가열대_분','균열대_분','총재로_분','인덕션히터후면_온도')
# 소수점(1f)_속도: y값으로 사용할 열들을 추가('STD_SPEED_8','RSB_최종속도','RSM_SPEED')
# 소수점(1f)_냉각수: y값으로 사용할 열들을 추가('#9~10ROLL(LPM)','#9~10ROLL(%)', '500(m3/h)', '500(%)', 'HVROLL(%)', 'RSBROLL(Bar)', 'PF/BROLL(%)', 'NTMROLL(%)','RSM(LPM)', 'RSMROLL(%)')

# y축 범위를 설정하는 딕셔너리 (각 y값마다 범위를 설정)
y_lim_dict = {
    '균열대.1': (1240, 1300),
    '가열대.1': (1000, 1200),  # 다른 변수에 대해 y축 범위 설정
    #'가열대_분': (0, 240),  # 다른 변수에 대해 y축 범위 설정
    #'균열대_분': (0, 240),  # 다른 변수에 대해 y축 범위 설정
    #'총재로_분': (0, 240),  # 다른 변수에 대해 y축 범위 설정
    '인덕션히터후면_온도': (900,1200),  # 다른 변수에 대해 y축 범위 설정
}

# 각 y값에 대해 그래프를 저장할 폴더 생성 및 그래프 생성
for y_col in y_columns:
    # y값 이름으로 폴더 생성
    output_folder = f'combined_graphs_{y_col.replace("/","_")}'
    os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성

    # 각 필터 조건에 대해 그래프 생성
    for condition in unique_conditions:
        filtered_a_df = merged_df[merged_df['지시외경'] == condition]  # A 그래프 데이터 필터링
        filtered_b_df = grouped[grouped['외경'] == condition]  # B 그래프 데이터 필터링

        if not filtered_a_df.empty and not filtered_b_df.empty:  # 두 데이터가 모두 있을 때만 그래프 그리기
            # '압연년월'을 문자열로 변환하여 정렬
            filtered_b_df = filtered_b_df.sort_values(by='압연년월')

            fig, ax1 = plt.subplots(figsize=(12, 6))

            # A 그래프 그리기 (왼쪽 y축, y_col에 따라 y값 자동 변경)
            sns.boxplot(x='year_month', y=y_col, data=filtered_a_df, ax=ax1, color='lightgray')
            ax1.set_xlabel('압연년월')
            ax1.set_ylabel(f'{y_col}', color='gray')
            ax1.tick_params(axis='y', labelcolor='gray')
            plt.xticks(rotation=90)

            # 그리드 추가
            plt.grid(True, color='gray',linestyle='--', linewidth=0.4)  # y축 기준으로 그리드 추가

            # A 그래프 중앙값 표시
            medians = filtered_a_df.groupby('year_month')[y_col].median()
            for xtick, median in zip(ax1.get_xticks(), medians):
                ax1.text(xtick, median, f'{median:.0f}', 
                        ha='center', va='bottom', color='black', fontsize=6)
                
            # A 그래프의 y축 범위 설정
            if y_col in y_lim_dict:
                ax1.set_ylim(y_lim_dict[y_col])

            # B 그래프 그리기 (오른쪽 y축)
            ax2 = ax1.twinx()  # 보조 y축 생성
            sns.lineplot(x='압연년월', y='불량율', data=filtered_b_df, ax=ax2, marker='o', color='darkred')
            ax2.set_ylabel('불량율 (%)', color='darkred')
            ax2.tick_params(axis='y', labelcolor='darkred')

            # B 그래프 y축 최소값을 0으로 설정
            ax2.set_ylim(0, None)  # y축 최소값을 0으로, 최대값은 자동 설정

            # 불량률 값 표시
            for index, row in filtered_b_df.iterrows():
                ax2.text(row['압연년월'], row['불량율'], f"{row['불량율']:.1f}", 
                        color='darkred', ha='center', va='bottom',fontsize=6)

            # 그래프 제목 및 레이아웃 설정
            plt.title(f'Combined Graph for 외경 {condition}')

            # x축 레이블이 잘리지 않도록 여백 조정
            plt.subplots_adjust(bottom=0.3)  # 아래쪽 여백을 늘려 x축 레이블 공간 확보
            
            # 그래프 전체 레이아웃 조정 (잘림 방지)
            fig.tight_layout()

            # 범례 추가
            #ax1.legend([f'{y_col}'], loc='upper left', frameon=False)
            #ax2.legend(['불량율(%)'], loc='upper right', frameon=False)

            # 그래프 저장
            plt.savefig(os.path.join(output_folder, f'combined_graph_{condition}_{y_col.replace("/","_")}.png'), dpi=300)
            plt.close()  # 그래프를 닫아 메모리 관리

# 완료 메시지
print("모든 그래프가 저장되었습니다.")

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
C:\Users\seah\AppData\Local\Temp\ipykernel_26444\3980097768.py:90: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()
C:\Users\seah\AppData\Local\Temp\ipykernel_26444\3980097768.py:90: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()
C:\Users\seah\AppData\Local\Temp\ipykernel_26444\3980097768.py:90: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()
C:\Users\seah\AppData\Local\Temp\ipykernel_26444\3980097768.py:90: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()
posx and p

모든 그래프가 저장되었습니다.
